In [1]:
import pandas_gbq


In [3]:
with open('app/SQL/Query_traer_clientes_simplificado.sql') as file:
    query=file.read()
data=pandas_gbq.read_gbq(query, project_id='customer-experience-384423',progress_bar_type=None)

c:\Users\jpmva\.virtualenvs\ETL---customer-experience-project-c9QD_7vZ\lib\site-packages\google\cloud\bigquery\table.py:2151: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [4]:
data

,id_cliente,promedio_compra,contactable,compro_en_OUTLET,compro_en_ESTELAR,compro_en_MOVIES,compro_en_LITTLE_MIC,compro_en_MOVIES_W,compro_en_OUTLET_MIC,compro_en_EVENTOS,...,compro_objeto_RELOJ,compro_objeto_MOCHILA,compro_objeto_ESTUCHES,compro_objeto_PANALERA,compro_objeto_DULCES,compro_objeto_CAMISETA,compro_objeto_SET,compro_objeto_STICKERS,compro_objeto_PANTALON,compro_objeto_ACCESORIOS
0,248896,117639.000000,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,148734,5629.400000,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,143936,39487.000000,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21281,86966.500000,SI,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,215527,54613.000000,SI,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250204,70708,20274.666667,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
250205,158522,38291.333333,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
250206,101430,29502.625000,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
250207,158457,19969.166667,NO,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
